In [ ]:
from time import process_time, time
import pandas as pd
from rocketpy import Environment, SolidMotor, Rocket, Flight
import concurrent.futures
from ConcurrentFunctions import runFlightWithMonteCarlo
from rocketpy.motors import motor
from IPython.display import display
from scipy.stats import norm
from rocketpy.mathutils.function import Function
from rocketpy.environment import environment

import FlightParams

%config InlineBackend.figure_formats = ['svg']
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline
mpl.rcParams["figure.figsize"] = [8, 5]
mpl.rcParams["figure.dpi"] = 120
mpl.rcParams["font.size"] = 14
mpl.rcParams["legend.fontsize"] = 14
mpl.rcParams["figure.titlesize"] = 14


import logging
import numpy as np

# Iterate the loop to read the cell values

env = Environment(latitude = FlightParams.envParams["latitude"], longitude = FlightParams.envParams["longitude"], elevation = FlightParams.envParams["elevation"])

analysis_parameters = {
    # Rocket's dry mass without motor (kg) and its uncertainty (standard deviation)
    "rocket_mass": (FlightParams.spMass, 0.001),
    # Rocket's inertia moment perpendicular to its axis (kg*m^2)
    "rocket_inertia_11": (FlightParams.spCentralDiameter, 0.036),
    # Rocket's inertia moment relative to its axis (kg*m^2)
    "rocket_inertia_33": (FlightParams.spCentralAxis, 0.00001),
    # Motors's dry mass without propellant (kg) and its uncertainty (standard deviation)
    "motor_dry_mass": (FlightParams.dryMotorMass, 0.001),
    # Motor's dry inertia moment perpendicular to its axis (kg*m^2)
    "motor_inertia_11": (FlightParams.motor_11_inertia, 0.016),
    # Motors's dry inertia moment relative to its axis (kg*m^2)
    "motor_inertia_33": (FlightParams.motor_33_inertia, 0.00001),
    # Distance between rocket's center of dry mass and motor's center of dry mass (m)

    # THIS is likely wrong, go back to check it

    "motor_dry_mass_position": (np.abs(FlightParams.the_motor_center_of_dry_mass_position - FlightParams.rocket_center_of_dry_mass_position), 0.001),
    # Motor total impulse (N*s)
    "impulse": (FlightParams.impulse, 25.3),

    "motor_position": (FlightParams.the_motor_position, .01),

    # Motor burn out time (s)
    "burn_time": (FlightParams.burn_time, .3),
    # Motor's nozzle radius (m)
    "nozzle_radius": (FlightParams.the_nozzle_radius, 0.5 / 1000),
    # Motor's nozzle throat radius (m)
    "throat_radius": (FlightParams.the_throat_radius, 0.5 / 1000),
    # Motor's grain separation (axial distance between two grains) (m)
    "grain_separation": (FlightParams.grainSeparation, 1 / 1000),
    # Motor's grain density (kg/m^3)
    "grain_density": (FlightParams.motor_density, 20),
    # Motor's grain outer radius (m)
    "grain_outer_radius": (FlightParams.grainOuterRadius, 0.3 / 1000),
    # Motor's grain inner radius (m)
    "grain_initial_inner_radius": (FlightParams.grainInnerRadius, 0.3 / 1000),
    # Motor's grain height (m)
    "grain_initial_height": (FlightParams.grainHeight, 1 / 1000),
    # Rocket's radius (kg*m^2)
    "radius": (FlightParams.spRadius, 0.001),
    # Distance between rocket's center of dry mass and nozzle exit plane (m) (negative)
    "nozzle_position": (FlightParams.rocket_center_of_dry_mass_position-(FlightParams.the_nozzle_position + FlightParams.the_motor_position), 0.001),
    # Distance between rocket's center of dry mass and and center of propellant mass (m) (negative)
    "grains_center_of_mass_position": (FlightParams.rocket_center_of_dry_mass_position-(FlightParams.the_motor_center_of_dry_mass_position + FlightParams.the_motor_position), 0.0001),

    # Multiplier for rocket's drag curve. Usually has a mean value of 1 and a uncertainty of 5% to 10%
    "power_off_drag": (FlightParams.power_off, 0.05),
    # Multiplier for rocket's drag curve. Usually has a mean value of 1 and a uncertainty of 5% to 10%
    "power_on_drag": (FlightParams.power_on, 0.05),
    # Axial distance between rocket's center of dry mass and nearest point in its nose cone (m)
    "nose_distance_to_CM": (FlightParams.rocket_center_of_dry_mass_position-FlightParams.nose_cone_length, 0.001),
    # Launch rail inclination angle relative to the horizontal plane (degrees)
    "inclination": (FlightParams.inclination, 1),
    # Launch rail heading relative to north (degrees)
    "heading": (FlightParams.heading, 2),
    # Drag coefficient times reference area for the drogue chute (m^2)
    "cd_s_drogue": (FlightParams.drogueCdS, 0.02),
    # Drag coefficient times reference area for the light chute
    "cd_s_light": (FlightParams.lightCdS, 0.02),
    # Time delay between parachute ejection signal is detected and parachute is inflated (s)
    "lag_rec": (FlightParams.lag_rec, 0.2),
    # Time delay between sensor signal is received and ejection signal is fired (s)
    "lag_se": (FlightParams.lag_se, 0.1),
    # temperature in kelvin
    "temperature": (FlightParams.kelvin_temp, 5),
    # atmospheric pressure in pascals
    "atmosphere_pressure": (env.pressure_ISA(FlightParams.envParams["elevation"]), 300),

    #without motor
    "rocket_CM": (FlightParams.rocket_center_of_dry_mass_position, 0.01),
}

# M1850W.all_info()

#sets up the ability to log inputs.
def setup_logging():
    class MyHandler(logging.FileHandler):
        def emit(self, record):
            super().emit(record)  # Call the superclass's emit method
            self.flush()  # Flush after each log message

    # Use your custom handler
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    handler = MyHandler('app.log', 'a')
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)

def createDistributionPlot(dispersonResultArray, name, lowerProbBound, upperProbBound, maxDecimals, percentConfidence, hypothesis, unit, makeGraph):
    mean = np.mean(dispersonResultArray)
    deviation = np.std(dispersonResultArray)
    unitString = ""
    if(unit == "meters"):
        unitString = "m"
    print(
        f'{name} -         Mean Value: {mean:0.3f} ' + unitString
    )
    print(
        f'{name} - Standard Deviation: {deviation:0.3f} m' + unitString
    )

    if(makeGraph):

        minimum = np.min(dispersonResultArray)
        maximum = np.max(dispersonResultArray)
        theBins = list()

        #using scott's rule to find the number of bins
        binWidth = 3.5 * deviation / np.power(len(dispersonResultArray), 1/3)
        numBins = int(np.round((maximum-minimum)/binWidth))

        changeInZ = (maximum - minimum) / numBins
        #test add
        zIterator = minimum
        for iterator in range(numBins):
            theBins.append(zIterator)
            zIterator += changeInZ
        finalValue = np.ceil(zIterator * np.power(10, maxDecimals)) / np.power(10, maxDecimals)
        theBins.append(finalValue)
        plt.figure()
        plt.xlim(minimum, maximum)
        minTextSize = 8
        maxTickCount = 50
        xTickBins = theBins
        if(numBins > maxTickCount):
            xTickBins = list()
            changeInZ = (maximum - minimum) / 50.0
            zIterator = minimum
            for iterator in range(50):
                xTickBins.append(zIterator)
                zIterator += changeInZ
        binListToString = list()
        for currentBin in xTickBins:
            binListToString.append(str(round(currentBin, maxDecimals)))
        plt.xticks(ticks = xTickBins, labels = binListToString, rotation = 75, fontsize = minTextSize)
        plt.hist(dispersonResultArray, bins=theBins)
        plt.title(name)
        plt.xlabel(name)
        plt.ylabel("Number of Occurences")

        plt.axvline(mean, color='k', linestyle='solid', linewidth=4)
        plt.axvline(mean - deviation, color='g', linestyle='solid', linewidth=3)
        plt.axvline(mean - 2*deviation, color='g', linestyle='solid', linewidth=3)
        plt.axvline(mean - 3*deviation, color='g', linestyle='solid', linewidth=3)
        plt.axvline(mean + deviation, color='g', linestyle='solid', linewidth=3)
        plt.axvline(mean + 2*deviation, color='g', linestyle='solid', linewidth=3)
        plt.axvline(mean + 3*deviation, color='g', linestyle='solid', linewidth=3)
        plt.show()
    lowerZ = (lowerProbBound - mean)/deviation
    upperZ = (upperProbBound - mean)/deviation
    probabilityInRange = norm.cdf(upperZ) - norm.cdf(lowerZ)
    print("PROBABILITY THE " + name +  " IS IN BETWEEN " + str(round(lowerProbBound, 4)) + " AND " + str(round(upperProbBound, 4)) + " IS: " + str(round(probabilityInRange * 100, 2)) + "%")
    if(percentConfidence != 100):
        aVal = (100 - percentConfidence)/100
        newZ = np.abs(norm.ppf(aVal/2))
        lowerBoundOfInterval = mean - newZ * deviation/np.sqrt(len(dispersonResultArray))
        upperBoundOfInterval = mean + newZ * deviation/np.sqrt(len(dispersonResultArray))
        print("There is a " + str(percentConfidence) + f"% chance that the mean of the " + name + 
              " falls within THIS RANGE: " + str(lowerBoundOfInterval) + ", " + str(upperBoundOfInterval))
        
        if(hypothesis != None):
            criticalValue = norm.ppf(aVal / 2)
            testStatistic = (mean - hypothesis) * np.sqrt(len(dispersonResultArray))/deviation
            print("\nWe have the hypothesis that our mean is greater than or equal to " + str(hypothesis) + ". With our critical percent, our critical value is " +
                str(criticalValue) + ". Then, our test statistic is " + str(testStatistic))
            if(testStatistic < criticalValue):
                print("Since our test statistic is more extreme than our critical value, we can reject the null hypothesis that the mean is " + str(hypothesis) + ".\n")
            else:
                print("Since our test statistic is less extreme than our critical value, we can NOT reject the null hypothesis that the mean is " + str(hypothesis) + ".\n")

currentRow = 1
# envParams["file"] = row[0].value

deploymentTimes = []
apogeeAtDeployment = []

#range only accepts integer values, so for now we make them integers and divide them later
for airbrakeDeployTimeAfterBurnOff in range (0, 20, 1):  
    print("THIS IS WHERE WE DEPLOY AIRBRAKE " + str(airbrakeDeployTimeAfterBurnOff) + " SECONDS AFTER BURNOUT")
    analysis_parameters["time_to_deploy_airbrake_after_burnout"] = (airbrakeDeployTimeAfterBurnOff, 0)
    deploymentTimes.append(airbrakeDeployTimeAfterBurnOff)

    setup_logging()
    # env.all_info()
    # Create data files for inputs, outputs and error logging
    filename = "MonteCarlo" + str(airbrakeDeployTimeAfterBurnOff)
    MonteCarloErrorFile = open("CreatedFiles/" + str(filename) + ".Errors.txt", "w")
    MonteCarloInputFile = open("CreatedFiles/" + str(filename) + ".Input.txt", "w")
    MonteCarloOutputFile = open("CreatedFiles/" + str(filename) + ".Output.txt", "w")

    initial_wall_time = time()
    initial_cpu_time = process_time()
    # Iterate over flight settings
    out = display("Starting", display_id=True)
    number_of_simulations = 2
    totalProcesses = 2
    simsPerProcess = number_of_simulations / totalProcesses

    start multiprocessing
    if __name__ == "__main__":
        with concurrent.futures.ProcessPoolExecutor(max_workers=totalProcesses) as executor:
            results = [executor.submit(runFlightWithMonteCarlo, simsPerProcess, FlightParams.envParams, analysis_parameters, initial_cpu_time, False) for _ in range(totalProcesses)] 
            for result in concurrent.futures.as_completed(results):
                MonteCarloInputFile.write(result.result()[0] + "\n")
                MonteCarloOutputFile.write(result.result()[1] + "\n")
                MonteCarloErrorFile.write(result.result()[2] + "\n")

    ## Print and save total time
    final_string = f"Completed {number_of_simulations} iterations successfully. Total CPU time: {process_time() - initial_cpu_time} s. Total wall time: {time() - initial_wall_time} s"
    out.update(final_string)
    MonteCarloInputFile.write(final_string + "\n")
    MonteCarloOutputFile.write(final_string + "\n")
    MonteCarloErrorFile.write(final_string + "\n")

    ## Close files
    MonteCarloInputFile.close()
    MonteCarloOutputFile.close()
    MonteCarloErrorFile.close()

    # Initialize variable to store all results
    dispersion_general_results = []

    dispersion_results = {
        "out_of_rail_time": [],
        "out_of_rail_velocity": [],
        "apogee_time": [],
        "apogee_altitude": [],
        "apogee_x": [],
        "apogee_y": [],
        "impact_x": [],
        "impact_y": [],
        "initial_static_margin": [],
        "out_of_rail_static_margin": [],
        "out_of_rail_stability_margin": [],
        "max_velocity": [],
        "execution_time": [],
    }

    # Get all dispersion results
    # Get file
    dispersion_output_file = open("CreatedFiles/" + str(filename) + ".Output.txt", "r+")

    # Read each line of the file and convert to dict
    for line in dispersion_output_file:
        # Skip comments lines
        if line[0] != "{":
            continue
        # Eval results and store them
        flight_result = eval(line)
        dispersion_general_results.append(flight_result)
        for parameter_key, parameter_value in flight_result.items():
            dispersion_results[parameter_key].append(parameter_value)

    # Close data file
    dispersion_output_file.close()

    # Print number of flights simulated
    N = len(dispersion_general_results)
    print("Number of simulations: ", N)

    # Retrieve dispersion data por apogee and impact XY position
    apogee_x = np.array(dispersion_results["apogee_x"])
    apogee_y = np.array(dispersion_results["apogee_y"])

    createDistributionPlot(dispersion_results["out_of_rail_stability_margin"], "Out Of Rail Stability Margin", lowerProbBound = 1.5, upperProbBound = 4, maxDecimals = 4, percentConfidence = 90, hypothesis = None, unit = "none", makeGraph = False)
    print("\n\n")
    createDistributionPlot(dispersion_results["apogee_altitude"], "Apogee Altitude", lowerProbBound = 9500 * 0.3048, upperProbBound = 10500 * 0.3048, maxDecimals = 0, percentConfidence = 90, hypothesis = 3048, unit = "meter", makeGraph = True)
    print("\n\n\n----------------------------------------------------------------------------------------------------\n\n\n")

    apogeeAtDeployment.append(np.mean(dispersion_results["apogee_altitude"]))


    #Calculate and plot landing elipses
    # Import libraries
    from imageio import imread
    from matplotlib.patches import Ellipse

    #img = imread("background.jpeg")

    # Retrieve dispersion data por apogee and impact XY position
    impact_x = np.array(dispersion_results["impact_x"])
    impact_y = np.array(dispersion_results["impact_y"])

    print(f"Simulated Landing Position x = {np.mean(dispersion_results['impact_x'])}, y = {np.mean(dispersion_results['impact_y'])}")

    # Define function to calculate eigen values
    def eigsorted(cov):
        vals, vecs = np.linalg.eigh(cov)
        order = vals.argsort()[::-1]
        return vals[order], vecs[:, order]


    # Create plot figure
    plt.figure(num=None, figsize=(8, 6), dpi=150, facecolor="w", edgecolor="k")
    ax = plt.subplot(111)

    # Calculate error ellipses for impact
    impactCov = np.cov(impact_x, impact_y)
    impactVals, impactVecs = eigsorted(impactCov)
    impactTheta = np.degrees(np.arctan2(*impactVecs[:, 0][::-1]))
    impactW, impactH = 2 * np.sqrt(impactVals)

    # Draw error ellipses for impact
    impact_ellipses = []
    for j in [1, 2, 3]:
        impactEll = Ellipse(
            xy=(np.mean(impact_x), np.mean(impact_y)),
            width=impactW * j,
            height=impactH * j,
            angle=impactTheta,
            color="black",
        )
        impactEll.set_facecolor((0, 0, 1, 0.2))
        impact_ellipses.append(impactEll)
        ax.add_artist(impactEll)

    # Calculate error ellipses for apogee
    apogeeCov = np.cov(apogee_x, apogee_y)
    apogeeVals, apogeeVecs = eigsorted(apogeeCov)
    apogeeTheta = np.degrees(np.arctan2(*apogeeVecs[:, 0][::-1]))
    apogeeW, apogeeH = 2 * np.sqrt(apogeeVals)

    # Draw error ellipses for apogee
    for j in [1, 2, 3]:
        apogeeEll = Ellipse(
            xy=(np.mean(apogee_x), np.mean(apogee_y)),
            width=apogeeW * j,
            height=apogeeH * j,
            angle=apogeeTheta,
            color="black",
        )
        apogeeEll.set_facecolor((0, 1, 0, 0.2))
        ax.add_artist(apogeeEll)

    # Draw launch point
    plt.scatter(0, 0, s=30, marker="*", color="black", label="Launch Point")
    # Draw apogee points
    plt.scatter(
        apogee_x, apogee_y, s=5, marker="^", color="green", label="Simulated Apogee"
    )
    # Draw impact points
    plt.scatter(
        impact_x, impact_y, s=5, marker="v", color="blue", label="Simulated Landing Point"
    )

    plt.legend()

    # Add title and labels to plot
    ax.set_title(
        "1$\sigma$, 2$\sigma$ and 3$\sigma$ Dispersion Ellipses: Apogee and Lading Points"
    )
    ax.set_ylabel("North (m)")
    ax.set_xlabel("East (m)")

    # Add background image to plot
    # You can translate the basemap by changing dx and dy (in meters)
    dx = 0
    dy = 0
    #plt.imshow(img, zorder=0, extent=[-1000 - dx, 1000 - dx, -1000 - dy, 1000 - dy])

    plt.axhline(0, color="black", linewidth=0.5)
    plt.axvline(0, color="black", linewidth=0.5)
    plt.xlim(-5000, 5000)
    plt.ylim(-5000, 5000)

    # Save plot and show result
    plt.savefig("CreatedFiles/" + str(filename) + ".pdf", bbox_inches="tight", pad_inches=0)
    plt.savefig("CreatedFiles/" + str(filename) + ".svg", bbox_inches="tight", pad_inches=0)
    plt.show()
    print("\n\n\n----------------------------------------------------------------------------------------------------\n\n\n")

plt.scatter(deploymentTimes, apogeeAtDeployment)
plt.title("Apogees at Different Deployment Times")
plt.xlabel("Airbrake Deployment Time (Seconds after Burnout)")
plt.ylabel("Apogee for that Time")
plt.show()

<>:386: SyntaxWarning: invalid escape sequence '\s'
<>:386: SyntaxWarning: invalid escape sequence '\s'
C:\Users\mahd\AppData\Local\Temp\ipykernel_24788\2471701608.py:386: SyntaxWarning: invalid escape sequence '\s'
  "1$\sigma$, 2$\sigma$ and 3$\sigma$ Dispersion Ellipses: Apogee and Lading Points"


THIS IS WHERE WE DEPLOY AIRBRAKE 0 SECONDS AFTER BURNOUT


'Completed 2 iterations successfully. Total CPU time: 4.84375 s. Total wall time: 4.929722785949707 s'

The wind speed at altitude 10.0m is 0.0m/s East
The wind speed at altitude 20.0m is 0.0m/s East
The wind speed at altitude 40.0m is 0.0m/s East
The wind speed at altitude 60.0m is 0.0m/s East
The wind speed at altitude 80.0m is 0.0m/s East
The wind speed at altitude 100.0m is 0.0m/s East
The wind speed at altitude 120.0m is 0.0m/s East
The wind speed at altitude 140.0m is 0.0m/s East
The wind speed at altitude 160.0m is 0.0m/s East
The wind speed at altitude 180.0m is 0.0m/s East
The wind speed at altitude 200.0m is 0.0m/s East
The wind speed at altitude 220.0m is 0.0m/s East
The wind speed at altitude 240.0m is 0.0m/s East
The wind speed at altitude 260.0m is 0.0m/s East
The wind speed at altitude 280.0m is 0.0m/s East
The wind speed at altitude 300.0m is 0.0m/s East
The wind speed at altitude 320.0m is 0.0m/s East
The wind speed at altitude 340.0m is 0.0m/s East
The wind speed at altitude 360.0m is 0.0m/s East
The wind speed at altitude 380.0m is 0.0m/s East
The wind speed at altitud

c:\Users\mahd\Documents\SimulationRocketPy\rocketpy\simulation\flight.py:1239: UserWarning: time_overshoot has been set to False due to the presence of controllers or sensors. 
  warnings.warn(


NOT GOOD TO DEPLOY, TIME IS: 0
NOT GOOD TO DEPLOY, TIME IS: 0
NOT GOOD TO DEPLOY, TIME IS: 1.001065225736757
NOT GOOD TO DEPLOY, TIME IS: 1.001065225736757
NOT GOOD TO DEPLOY, TIME IS: 2.0523460067728747
NOT GOOD TO DEPLOY, TIME IS: 2.0523460067728747
NOT GOOD TO DEPLOY, TIME IS: 3.0018725238479496
NOT GOOD TO DEPLOY, TIME IS: 3.0018725238479496
NOT GOOD TO DEPLOY, TIME IS: 4.001752352945075
NOT GOOD TO DEPLOY, TIME IS: 4.001752352945075
NOT GOOD TO DEPLOY, TIME IS: 5.435577906581928
NOT GOOD TO DEPLOY, TIME IS: 5.435577906581928
NOT GOOD TO DEPLOY, TIME IS: 6.684124791593733
NOT GOOD TO DEPLOY, TIME IS: 6.684124791593733
WE GOOD TO DEPLOY WE BALL GRAHHHHH: 8.283725514211016
WE GOOD TO DEPLOY WE BALL GRAHHHHH: 8.283725514211016
WE GOOD TO DEPLOY WE BALL GRAHHHHH: 9.79054753573578
WE GOOD TO DEPLOY WE BALL GRAHHHHH: 9.79054753573578
WE GOOD TO DEPLOY WE BALL GRAHHHHH: 12.27865181035915
WE GOOD TO DEPLOY WE BALL GRAHHHHH: 12.27865181035915
WE GOOD TO DEPLOY WE BALL GRAHHHHH: 16.276111517

C:\Users\mahd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mahd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\mahd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\mahd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encounte

ValueError: zero-size array to reduction operation minimum which has no identity